In [13]:
#import torch
#from transformers import pipeline
from glob import glob
import pandas as pd 
import numpy as np
from nltk import sent_tokenizer

ModuleNotFoundError: No module named 'nltk'

In [12]:
paths = glob('data/Subtitles/*.ass')[:5]

In [18]:
with open(paths[0], 'r') as file:
    lines = file.readlines()
    lines = lines[27:]
    script = ' '.join([','.join(line.split(',')[9:]).strip() for line in lines])

script = script.replace('\\N', " ")

eposide_name = paths[0].split('-')[-1].split('.')[0].strip()
print(eposide_name, "\n", script)

01 
 A long time ago, a powerful demon fox appeared with nine tails. With its powerful tails, it could smash mountains and create tidal waves. A band of Ninjas rose to defend their village from attack. We have to wait until the Fourth Hokage gets here! We can't let it get any closer to our village! One great Ninja was able to imprison the monster, but died in the process. This Ninja was known as… the Fourth Hokage. Naruto! Why did you do such a thing?! You're really gonna get it this time! I don't care! You know your problem? You can't do the things I do! Only I can do this! I'm better than all of you! Believe it! There's a problem, sir! Lord Hokage! What is it? Did that Naruto do something again? Yes. He climbed onto the Mountainside Images… And he vandalized and graffitied all over them! Wait! Ha ha… Why should I? Hey, Naruto! How did you suddenly get here, lruka Sensei? The question is what are you doing here when you should be in class now? Now listen, Naruto. You failed the last g

In [ ]:
def read_data(paths):
    scripts = []
    eposide_names = []
    for path in paths:
        with open(path, 'r') as file:
            lines = file.readlines()
            lines = lines[27:]
            script = ' '.join([','.join(line.split(',')[9:]).strip() for line in lines])

        script = script.replace('\\N', " ")
        eposide_name = int(path.split('-')[-1].split('.')[0].strip())
        scripts.append(script)
        eposide_names.append(eposide_name)

    return pd.DataFrame.from_dict({"eposide": eposide_names, "script": scripts})

In [20]:
paths = glob('data/Subtitles/*.ass')
df = read_data(paths)
df.head()

,eposide,script
0,01,"A long time ago, a powerful demon fox appeared..."
1,02,"C'mon! Running like a fugitive, Being chased b..."
2,03,"C'mon! Running like a fugitive, Being chased b..."
3,04,"C'mon! Running like a fugitive, Being chased b..."
4,05,"C'mon! Running like a fugitive, Being chased b..."


In [ ]:
def load_model(device):
    classifier = pipeline("zero-shot-classification",
                      model="facebook/bart-large-mnli",
                      device=device)
    return classifier

In [ ]:
def script_to_batches(script):
    script_sentense = sent_tokenizer(script)
    bathch_size = 20
    script_batch = []
    for index in range(0, len(script_sentense), bathch_size):
        batch = script_sentense[index:index+bathch_size]
        script_batch.append(batch)
    return script_batch

def get_labels_df(df):
    for i in len(df):
      script_row = df.iloc[i]['script']
      script_batches = script_to_batches(script_row)
      labels, scores = classify_batches(script_batches)


In [ ]:
def classify_batches(script_batches):
    labels = []
    outputs = classifier(script_batches, labels, multi_label=True)
    theme = {}
    for cell in outputs:
        for label, score in zip(cell['labels'], cell['scores']):
            if label not in theme:
                theme[label] = []
            theme[label].append(score)
    output = {}
    output = {key: np.mean(np.array(value)) for key, value in theme.items()}
    return output

In [ ]:
device = 'cuda' if torch.cuda.is_avilable() else 'cpu'
classifier = load_model(device)